
<a href="https://colab.research.google.com/github/isikus/MachineScorning/blob/main/Нагенерировать%20машинного%20отупения%20на%20ruGPT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Нагенерировать машинного отупения на ruGPT3

In [1]:
import os
import json

def is_empty(fpath, default_lines=[["Дополни эту фразу", "И эту фразу тоже дополни"], ["Первый пример запроса", "Из нескольких строк", "", "И второй пример", "Я тоже придумать смог"]]):
  if not os.path.isfile(fpath):
    return True
  if not (os.path.getsize(fpath) > 0):
    return True
  if default_lines:
    with open(fpath, "r") as infile:
      lines = infile.read().split("\n")
    for deflines in default_lines:
      if lines == deflines:
        return True
  return False
  

def get_lbc(model_meta):
  lbc = model_meta["line_break_character"]
  lbc = lbc.replace("\s", " ")
  return lbc

#@markdown Для начала выберите модель:

Модель = "Нейроюморески"  #@param ["Нейроописания товаров", "Нейроинструкции на все случаи жизни", "Нейротексты «Короля и Шута»", "Нейроновости Северной Кореи", "Нейропоздравления с новым годом", "Нейроюморески"]

desc_dict = {
    "Нейроописания товаров": "products_desc",
    "Нейроинструкции на все случаи жизни": "wikihowto",
    "Нейротексты «Короля и Шута»": "kishverses",
    "Нейроновости Северной Кореи": "rukcna",
    "Нейропоздравления с новым годом": "greetings",
    "Нейроюморески": "neurojumoreski"
}

model = desc_dict[Модель]

!wget -q --show-progress https://storage.googleapis.com/ml-bucket-isikus/machine-scorning/{model}.tar.gz
!mkdir {model}
!tar -xzf {model}.tar.gz -C {model}

try:
  with open(model+"/meta.json", "r") as injson:
    model_meta = json.load(injson)
  print('\x1b[1m'+Модель + ": модель подготовлена"+'\x1b[0m')
except Exception as e:
  print('\x1b[1;31m'+"Что-то сломалось – модель не готова :("+'\x1b[0m')

neurojumoreski.tar. 100%[===================>] 443.70M   113MB/s    in 4.4s    
Нейроюморески: модель подготовлена


Теперь запишите варианты начала сообщений, которые должна дополнить модель, сразу под текстом `%%writefile input.txt`, по 1 разному началу на строку:

In [3]:
%%writefile input.txt
Дополни эту фразу
И эту фразу тоже дополни

Writing input.txt


**Важно:** если ваши запросы содержат переносы строк (например, это начало стихотворения), вместо предыдущей клетки, заполните клетку ниже, разбивая ваш ввод двумя переносами строк. Код ниже заменит переносы строк на специальный символ – это необходимо для корректной работы модели.

In [4]:
%%writefile input_with_lines.txt
Первый пример запроса
Из нескольких строк

И второй пример
Я тоже придумать смог

Writing input_with_lines.txt


Если у вас совсем нет идей, запустите код ниже – он скачает примеры запросов по умолчанию для вашей модели:

In [5]:
if is_empty("input.txt") and is_empty("input_with_lines.txt"):
  !cp {model}/example_input.txt input.txt
  print('\x1b[1;31m'+"Загружены примеры по умолчанию"+'\x1b[0m')

Загружены примеры по умолчанию


In [6]:
#@markdown Наконец, укажите, сколько вариантов выдачи нужно сгенерировать модели на каждый из запросов (от 1 до 30):

Повторов = 4  #@param {"type": "integer"}
takes = Повторов

if takes < 1:
  takes = 1
  print("Это маловато. Установили значение " + str(takes) + ".")
if takes > 30:
  takes = 30
  print("Это многовато. Установили значение " + str(takes) + ".")
if type(takes) != int:
  takes = 4
  print("Что-то пошло не так. Установили значение " + str(takes) + ".")

## Подготавливаем среду для модели
Сами файлы скачаны, но нужно ещё выполнить код, чтобы подготовить модель к работе.

Проверяем, что мы подключены к видеокарте: команда в клетке ниже должна работать и показывать показывать примерно 12000-15000 Мб свободной памяти:

In [7]:
!nvidia-smi

Sat Mar  6 11:30:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Устанавливаем нужные версии библиотек:

In [8]:
!pip3 install -q urllib3==1.25.4 transformers==2.8.0

print('\n'+'\x1b[1m'+"Требуемые версии пакетов установлены"+'\x1b[0m')

     |████████████████████████████████| 133kB 9.4MB/s 
     |████████████████████████████████| 573kB 17.5MB/s 
     |████████████████████████████████| 5.6MB 16.7MB/s 
     |████████████████████████████████| 1.2MB 52.0MB/s 
     |████████████████████████████████| 890kB 45.0MB/s 
     |████████████████████████████████| 133kB 58.7MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 7.3MB 54.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.

Требуемые версии пакетов установлены


Скачиваем файл с исходным кодом для работы модели:

In [9]:
!wget -q --show-progress https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

print('\n'+'\x1b[1m'+"Код запуска модели загружен"+'\x1b[0m')

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      

Код запуска модели загружен


Запускаем код, необходимый для преобразования файлов и отображения выдачи:

In [10]:
import re
import os
import sys
import json
import random
from datetime import datetime

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

with open(model+"/meta.json", "r") as injson:
  model_meta = json.load(injson)

# converting inputs

s = ""

if not is_empty("input_with_lines.txt"):
  with open("input_with_lines.txt", "r", encoding="utf-8") as intxt:
    prompts = [get_lbc(model_meta).join([sp for sp in p.split("\n") if sp]) for p in intxt.read().split("\n\n") if p]
else:
  with open("input.txt", "r", encoding="utf-8") as intxt:
    prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * takes

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

print('\x1b[1m'+"Среда подготовлена"+'\x1b[0m')

Среда подготовлена


## Генерируем отупение

Теперь всё, наконец, готово к тому, чтобы модель сгенерировала нам выдачу. То, насколько серьёзный и осмысленный текст будет пытаться выдавать модель, можно регулировать по шкале **`Бредовость`**: здесь есть 5 степеней – от самой нормальной до самой странной (я предупреждал). Кроме этого, флажок **`Повторяться`** определяет, как будет работать модель: если галочку оставить, то первый результат будет очень похож не те, что есть в канале, а последующие будут с ним совпадать. А если снять галочку, то выдача модели каждый раз будет новая. Удачи!

In [47]:
#@markdown ### Начать генерацию!

Бредовость = 75 #@param {type: "slider", min: 0, max: 100, step: 25}
stlv = Бредовость

stup_cases = {
    0: {"k": 5, "p": 0.95, "temp": 1.0},
    25: {"k": 5, "p": 0.93, "temp": 2.0},
    50: {"k": 6, "p": 0.91, "temp": 3.0},
    75: {"k": 6, "p": 0.9, "temp": 4.0},
    100: {"k": 7, "p": 0.88, "temp": 5.0}
}

Повторяться = True   # @param {"type": "boolean"}
add_randomness = not Повторяться

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
printmd("Ключ случайного значения: **"+str(rs)+"**")

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k={str(stup_cases[stlv]["k"])} \
    --p={str(stup_cases[stlv]["p"])} \
    --seed={str(rs)} \
    --temperature={str(stup_cases[stlv]["temp"])} \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

resps = [r for r in responses.split("Context >>> ruGPT:\n") if r]
sps = [sp for sp in s.split("\n") if sp]
for sp, resp in zip(sps, resps):
  pt = re.search(re.sub(r"%[a-z]+", "(.*)", re.escape(model_meta["prompt"])), sp).group(1)
  rs, _ = re.findall(re.compile(re.escape(sp) + r"\n?(.*?)(" + ssc + "|$)", flags=re.DOTALL), resp)[0]
  pt = "<br>".join("<b>" + p + "</b>" for p in re.split(lbc, pt) if p)
  printmd(pt)
  rs = re.sub(lbc, "<br>", rs).strip()
  rs = re.sub(r"([^\s])<br>([^\s])", r"\1 <br> \2", rs)
  rse = len(rs) if rs.find("Context >>") == -1 else rs.find("Context >>")
  printmd(rs[:rse] + "<br>")

Ключ случайного значения: **5817**

**Идёт медведь по лесу, видит – машина горит.**

Он спрашивает у неё, мол что делать будем. Машина загорелась, медведь её догнал и давай еб*ать.<br>

**Идёт медведь по лесу, видит – машина горит.**

Сел в неё, а оттуда дым, гарь. Ну медведь думает: ща народ приедет расскажу, что тут творится, что делать буду... <br> В этот раз приезжает комиссия с мужскими долгами и говорит: <br>– Мужики давайте мы вас отвезём к нашим юристам, там решим, что с вами можно сделать, а что нет. Идёт медведь дальше и смотрит, а там машина стоит и дым идёт, гарный такой. Он говорит мужикам: <br> - А давайте я вас отвезу с юристу домой к нашим женам, пусть там думают, что делать. <br> Приезжает мужик в суд с иском, выигрывает его. Приезжает домой с тремя детьми: Сашенькой, Денькой, Илонкой и Леночкой, а ему говорит: <br> - А вот вам по маленькой. Сашок – большой, а Димка – большой! <br> Приехал мужик в гаи с жалобе. Жуткие разборчии: <br> — Ну, как там, что у вас? Что с деньгами стало?<br>

...

Первый заказывают три кружки пива. <br>- Сколько же мне взять этого напитка? — спрашивает бармен <br>Первый заказывает пять бутылок водки <br>второй бутылку шампанского. Первый заказывает пол бокала <br>а второму. Третий заказывает пол бутылки <br>виски, а второй половину 
 виски<br>

**Бесконечное число математиков заходит в бар.**

Один наливают виски другому, тот третий наливает себе третью, бармен смеется, у бармена из штанов капают капли и падает ему прямо между ног... <br> - Ёпты-ы, бляяяя-а!!!<br>

**Бесконечное число математиков заходит в бар.**

Один наливает им пинта бордо, другой выпив немного. И тут бармен, наклоня к одному из математиков рюмку и не отрываясь от работы:\t  <br>— Это кто тут так много нас? Это что же получается, что мы одни во вселенной!? Это что за хуйня?! Кто этот маленький вонючий коммунистический пидаpас и говоpит, что в его рюмке есть вода и моpылка!? Этот грязный коммунистического пидаpас пытается что-то там мне накритиковать, а мне говорит, мол давай мы ему скажем. А я отвечаю: давай ты покажешь мне свой настоящий талант, я покажу тебе, кто из людей, сидоров мудаков ебал... <br>— Ты что, ебанушка совсем, блять?! Ты че, в своем баре совсем одичал?! Ёбаный в рот, блядь!..
<br>

## Больше отупения!

Хотите попробовать другие фразы? Тогда впишите их в клетку под этим текстом, как до этого, и запустите код ниже. Не забудьте поменять `%%writefile input.txt` на `%%writefile input_with_lines.txt`, если ваши запросы содержат переносы строк!

In [44]:
%%writefile input.txt
 
 

Overwriting input.txt


In [45]:
#@markdown ### Генерируем новую выдачу

Повторов = 30  #@param {"type": "integer"}
takes = Повторов

if takes < 1:
  takes = 1
  print("Маловато повторов. Установлено значение " + str(takes) + ".")
if takes > 30:
  takes = 30
  print("Многовато повторов. Установлено значение " + str(takes) + ".")
if type(takes) != int:
  takes = 4
  print("Что-то пошло не так с количеством повторов. Установлено значение " + str(takes) + ".")

s = ""

if not is_empty("input_with_lines.txt"):
  with open("input_with_lines.txt", "r", encoding="utf-8") as intxt:
    prompts = [get_lbc(model_meta).join([sp for sp in p.split("\n") if sp]) for p in intxt.read().split("\n\n") if p]
else:
  with open("input.txt", "r", encoding="utf-8") as intxt:
    prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * takes

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

Бредовость = 50 #@param {type: "slider", min: 0, max: 100, step: 25}
stlv = Бредовость

Повторяться = False   # @param {"type": "boolean"}
add_randomness = not Повторяться

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
printmd("Ключ случайного значения: **"+str(rs)+"**")

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k={str(stup_cases[stlv]["k"])} \
    --p={str(stup_cases[stlv]["p"])} \
    --seed={str(rs)} \
    --temperature={str(stup_cases[stlv]["temp"])} \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

resps = [r for r in responses.split("Context >>> ruGPT:\n") if r]
sps = [sp for sp in s.split("\n") if sp]
for sp, resp in zip(sps, resps):
  pt = re.search(re.sub(r"%[a-z]+", "(.*)", re.escape(model_meta["prompt"])), sp).group(1)
  rs, _ = re.findall(re.compile(re.escape(sp) + r"\n?(.*?)(" + ssc + "|$)", flags=re.DOTALL), resp)[0]
  pt = "<br>".join("<b>" + p + "</b>" for p in re.split(lbc, pt) if p)
  printmd(pt)
  rs = re.sub(lbc, "<br>", rs).strip()
  rs = re.sub(r"([^\s])<br>([^\s])", r"\1 <br> \2", rs)
  rse = len(rs) if rs.find("Context >>") == -1 else rs.find("Context >>")
  printmd(rs[:rse] + "<br>")

Ключ случайного значения: **7582**

** **

1\sum\sin^4\left(1/2 x \frac{2 x^2 ^2} \right )$ <br>  /.\begin{algorithm*}\frametrach (1-(1-1/2))\end{algorithm*}<br>

** **

"Бляяядь. Я сейчас умру, мне не жить, бля. <br>- Ну вот, например: ты можешь говорить,  но никогда, НИЧЕГО! Это невозможно. <br>- Но ты можешь! <br>- Нет. <br>- Тогда почему ты <br>разговариваешь вообще? Ты же сам сказал - это не работает!<br>

...

ewpage=\jumoreski\jumoresk<br>

** **

ewpage:s>
❗Смешной случай произошел в детском садике. Учительница, заметив на дверях классного отдела надпись «Солнечный зайчик» и замечание заведующей, попросила ее объяснить: <br> — Что это такое и почему так? <br> Вежливо попросили выйти из кабинета, учительницу отпустили, зав. кабинетом предложила завуча и учительницу по коридору подойти и спросить: "Почему это ты так решил?" <br> Ученица первой ответила: <br> — Это надпись на моей классной машине. На капот, бамперах и радиаторной решётке написано. <br>— А где написано, почему так написано? <br> Учительница покраснела и, обращаясь к ученице, добавила: «Ты наверное еще маленькая и не поняла всей глубины проблемы. Если бы не твоя глупая шутка про «Снег на капоте» (а я в неё и так насмотрелся) я бы тебя давно расстрелял.» <br> После этой учительнице дали восемь суток за то, что она «разговаривает» на уроке в присутствии несовершеннолетних. <br>Ветеран ВОВ рассказывает в интервью "Новому Русскому", что был такой случай: <br> «У меня в школе учительница говорила, чтобы я обязательно зашел к учителям. Но когда я вошел на кухню, все дети на уроке уже разошлись. <br>Я подошел к окну и сел. Вдруг из моей комнаты доносится: «Кто здесь?». В это же самое время на меня кто-то напал с табуретками. В этот момент дверь полностью открывается и выходит женщина в короткой юбочке, резиновых сапогах и валенок на ногах». <br>Ветеран ВОВ не поверил, и отправил письмо в фонд помощи голодающим детям. Но ответа так и не получил. <br> "Здравствуйте" — спрашивает женщина у матери в надежде услышать ответ. <br> "Нет-нет" — отвечают ей — "Я просто подумал, что вы не хотите отвечать. Вы не ответили на наш последний вопрос". <br> "Я думаю, что я не смогу вас сейчас вам сказать. Вы знаете, мне нужно с кем- то посоветоваться, я не смогу сейчас отвечать с кем бы ни пришла к вам на разговор." — "Это не может быть правдой, вы же знаете — у каждого человека свои потребности."" <br> Женщина уходит от учителя и возвращается через некоторое время. <br>"Я только что пришла к вашему учителю и сказала ему, что я ухожу с вами на ужин. А вы сказали,что уедете"<br>

** **

♂Dungeon Master: Sexual Destruct <br>♂Sexual Destroyer: The Master <br>♂Godframe: The Master
<br>

На этом всё. Получайте удовольствие (надеюсь)!